In [8]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes
import numpy as np

In [20]:
L = 3
N = 3
t = 1.0
U = 1.0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

In [21]:
# [Part A] Set states and matrix representation of operators in local Hilbert space
site_basis, site_ops = [], []

for k in range(L):
    basis = [(Q(0), 1), (Q(1), 1), (Q(2), 1), (Q(3), 1)]
    ops = {
        "": np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]),             # identity
        "c": np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, np.sqrt(2), 0, 0], [0, 0, np.sqrt(3), 0]]),  # creation
        "d": np.array([[0, 1, 0, 0], [0, 0, np.sqrt(2), 0], [0, 0, 0, np.sqrt(3)], [0, 0, 0, 0]]),  # annihilation
    }
    site_basis.append(basis)
    site_ops.append(ops)

In [22]:
# [Part B] Set Hamiltonian terms
driver.initialize_system(n_sites=L, vacuum=Q(0), target=Q(N), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

b.add_term("cd", np.array([[i, i + 1, i + 1, i] for i in range(L - 1)]).flatten(), t)
b.add_term("ccdd", np.array([i for i in range(L) for _ in range(L)]), U/2)
b.add_term("cc", np.array([i for i in range(L) for _ in range(L)]), U/2)

# [Part C] Perform DMRG
mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=None), algo_type=MPOAlgorithmTypes.FastBipartite)

AssertionError: 

In [115]:
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=1)
driver.expectation(mps, mpo, mps)

4.941964249103701

In [116]:
b = driver.expr_builder()
b.add_term("cd", np.array([i for i in range(L) for _ in range(2)]), 1.0)
mponum = driver.get_mpo(b.finalize(adjust_order=True), algo_type=MPOAlgorithmTypes.FastBipartite)
driver.expectation(mps, mponum, mps)

9.999999999999996

In [117]:
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=1)
energy = driver.dmrg(
    mpo, 
    mps, 
    n_sweeps=100, 
    bond_dims=[50] * 4 + [100] * 2 + [200] * 2, 
    noises=[1e-3] * 3 + [1e-5] * 3 + [0],  
    thrds=[1e-10] * 8, 
    dav_max_iter=100, 
    cutoff=1E-20, 
    iprint=1)
print("DMRG energy = %20.15f" % energy)


Sweep =    0 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      0.701 | E =     -61.6864166094 | DW = 1.75484e-03

Sweep =    1 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      1.545 | E =     -61.0647590641 | DE = 6.22e-01 | DW = 1.39749e-02

Sweep =    2 | Direction =  forward | Bond dimension =   50 | Noise =  1.00e-03 | Dav threshold =  1.00e-10
Time elapsed =      2.436 | E =     -56.1004330522 | DE = 4.96e+00 | DW = 1.57994e-02

Sweep =    3 | Direction = backward | Bond dimension =   50 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      3.334 | E =     -63.3676492860 | DE = -7.27e+00 | DW = 1.70730e-02

Sweep =    4 | Direction =  forward | Bond dimension =  100 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      4.377 | E =     -60.4165501888 | DE = 2.95e+00 | DW = 2.72457e-04

Sweep =    5 | Direction = backward | B

KeyboardInterrupt: 

In [16]:
L = 3
np.array([j for i in range(L - 1) for j in [i, i + 1, i + 1, i]])

array([0, 1, 1, 0, 1, 2, 2, 1])

In [23]:
np.array([[i, i+1, i+1, i] for i in range(L-1)]).flatten()

array([0, 1, 1, 0, 1, 2, 2, 1])

In [26]:
np.repeat(np.arange(L-1), 4) + np.tile([0, 1, 1, 0], L-1)

array([0, 1, 1, 0, 1, 2, 2, 1])

In [27]:
np.repeat(np.arange(L-1), 4)

array([0, 0, 0, 0, 1, 1, 1, 1])

In [28]:
np.tile([0, 1, 1, 0], L-1)

array([0, 1, 1, 0, 0, 1, 1, 0])

In [17]:
np.array([i for i in range(L)])

array([0, 1, 2])

In [24]:
np.array(np.arange(L))

array([0, 1, 2])

In [18]:
np.array([j for i in range(L) for j in [i, i]])

array([0, 0, 1, 1, 2, 2])

In [25]:
np.repeat(np.arange(L), 2)

array([0, 0, 1, 1, 2, 2])